# Unfairness Detection Notebook


--------------------------
### Dataset

In this ML pipeline, we will try to find unfairness, and biased classifications.

The dataset is the famous **adult** dataset from [Kaggel](https://www.kaggle.com/datasets/wenruliu/adult-income-dataset) (Kaggle website, 2023).

----------------------
### The problem

After data explorarion and cleaning, we will do classification and find out if we have direct or indirect unfairness in the model using definitions of **disarate imapct** , **disparate treatment** , and **disparate mistreatment**. Also using **KNN classification** specifically to detect it.

There is a sensitive feature in the datset, **sex** and the target label is whether people have an income which is higher than $50K or lower.

In [1]:
import google.colab
import io
import IPython.display
import PIL

import pandas
import numpy

import sklearn.model_selection
import sklearn.metrics
import sklearn.svm
import sklearn.neighbors
import sklearn.tree
import imblearn.under_sampling
import sklearn.preprocessing
import sklearn.compose

## Data Collection

In [5]:
uploaded = google.colab.files.upload()
adult = pandas.read_csv(io.BytesIO(uploaded['adult.csv']))

Saving adult.csv to adult.csv


In [6]:
adult.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [7]:
adult.shape

(32561, 15)

## Data Exploration, Preprocessing, and Feature Engineering

Splitting the data in the beginning to have unseen sets for the testing and avoiding data leakage.

In [8]:
train, test = sklearn.model_selection.train_test_split(adult, test_size = 0.2)

In [9]:
test.shape

(6513, 15)

In [10]:
train.shape

(26048, 15)

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26048 entries, 15416 to 21944
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             26048 non-null  int64 
 1   workclass       26048 non-null  object
 2   fnlwgt          26048 non-null  int64 
 3   education       26048 non-null  object
 4   education.num   26048 non-null  int64 
 5   marital.status  26048 non-null  object
 6   occupation      26048 non-null  object
 7   relationship    26048 non-null  object
 8   race            26048 non-null  object
 9   sex             26048 non-null  object
 10  capital.gain    26048 non-null  int64 
 11  capital.loss    26048 non-null  int64 
 12  hours.per.week  26048 non-null  int64 
 13  native.country  26048 non-null  object
 14  income          26048 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.2+ MB


In [12]:
train.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
income            0
dtype: int64

There are no null values in the dataset, but we can see above that there are "?" values in the head of the dataset shown above. we explore more to find out how is our dataset and what needs to be done.


Some features are categorical, so we have to encode them before feeding them to the model.

we need to eliminate the duplicate data. specifically the rows that are identical here.

In [13]:
train = train.drop_duplicates()

In [14]:
train.shape

(26031, 15)

In [15]:
value_list = [train[col_name].unique() for col_name in train.columns]
value_list

[array([32, 42, 20, 34, 37, 56, 27, 17, 44, 25, 26, 38, 24, 29, 33, 45, 23,
        40, 53, 52, 55, 79, 28, 51, 57, 19, 67, 43, 41, 50, 21, 39, 36, 69,
        54, 35, 47, 48, 62, 18, 30, 46, 71, 70, 90, 68, 60, 63, 31, 22, 66,
        61, 78, 76, 58, 77, 73, 81, 49, 65, 72, 74, 64, 59, 80, 75, 84, 88,
        82, 83, 85, 87]),
 array(['Private', 'State-gov', 'Local-gov', '?', 'Self-emp-inc',
        'Self-emp-not-inc', 'Federal-gov', 'Without-pay', 'Never-worked'],
       dtype=object),
 array([245487,  55764, 278155, ..., 146788, 228190, 327769]),
 array(['5th-6th', 'Assoc-acdm', 'Some-college', 'Bachelors', 'HS-grad',
        '11th', 'Assoc-voc', '10th', 'Doctorate', '12th', 'Masters', '9th',
        'Prof-school', '1st-4th', '7th-8th', 'Preschool'], dtype=object),
 array([ 3, 12, 10, 13,  9,  7, 11,  6, 16,  8, 14,  5, 15,  2,  4,  1]),
 array(['Married-civ-spouse', 'Divorced', 'Never-married',
        'Married-spouse-absent', 'Widowed', 'Separated',
        'Married-AF-spouse'], d

to encode columns from categorical to a numerical, we put the names of categorical columns with **more than two values** and the numerical ones in separate lists. We replace the ones that have only *two values*, with numeric ones manually, it is easy.

*We can also eliminate the *education* column since it has another identical column with numeric values: **edcuation.num** *

In [16]:
categorical = ["workclass",
               "marital.status",
               "occupation",
               "relationship",
               "race",
               "native.country"
               ]

In [17]:
numerical = ["age",
             "fnlwgt",
             "education.num",
             "capital.gain",
             "capital.loss",
             "hours.per.week"
            ]

In [18]:
train = train.drop(columns = ("education"), axis = 1)

In [19]:
no_nun = train.replace("?", numpy.nan)
no_nun.isnull().sum()

age                  0
workclass         1468
fnlwgt               0
education.num        0
marital.status       0
occupation        1475
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     458
income               0
dtype: int64

This ML pipeline is set on finding unfairness. That is why I have chosen to eliminte the missing data and not have any unreal data. so instead of replacing "?" with values like the mode of the column, I replaced it with null values to eliminate them in spite of the fact that we are losing 7% of the rows.

In [20]:
train = no_nun.dropna()
train.shape

(24118, 14)

Using *value_counts* we can see how many of each value we have in our target (which is the **income**).

this is to see if we have a balanced datset or not.

In [21]:
train["income"].value_counts()

<=50K    18131
>50K      5987
Name: income, dtype: int64

The dataset is imbalanced already, the ratio is roughly 3 to 1 for the people with less than $ 50,000 income and the other ones above this amount. But instead of using oversampling (to have more data) and generate fake data, I prefer to do undersampling and make the data balanced and more fair to begin with and have a better trained model.

first we'd better replace categorical data with numeric ones in the target label and sex column, which have only two categorical values.

In [22]:
sex = {"Male" : 1, "Female" : 0}
train["sex"].replace(to_replace = sex, inplace = True)
train["sex"].unique()

<ipython-input-22-a49ef4a81f11>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["sex"].replace(to_replace = sex, inplace = True)


array([1, 0])

In [23]:
income = {">50K" : 1, "<=50K" : 0}
train["income"].replace(to_replace = income, inplace = True)
train["income"].value_counts()

<ipython-input-23-4154b435e815>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["income"].replace(to_replace = income, inplace = True)


0    18131
1     5987
Name: income, dtype: int64

### Undersampling to have a balanced dataset

In [24]:
x_train = train.drop(columns = ("income"), axis = 1)
y_train = train["income"]

In [25]:
under_class = imblearn.under_sampling.RandomUnderSampler()
x_resampled, y_resampled = under_class.fit_resample(x_train, y_train)
x_resampled.shape

(11974, 13)

This is alomst half of the orginal train set that we had.

Applying necessary changes to the **test set**

In [26]:
test = test.drop_duplicates()
test = test.drop(columns = ("education"), axis = 1)

no_nun_test = test.replace("?", numpy.nan)
test = no_nun_test.dropna()

test["sex"].replace(to_replace = sex, inplace = True)
test["income"].replace(to_replace = income, inplace = True)

x_test = test.drop(columns = ("income"), axis = 1)
y_test = test["income"]

<ipython-input-26-d9d26ff4ee0b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["sex"].replace(to_replace = sex, inplace = True)
<ipython-input-26-d9d26ff4ee0b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["income"].replace(to_replace = income, inplace = True)


### Applying **One Hot Encoder** to the categorical features.

our catogircal features are nor ordered so we use this, since the Lable encoder should only be used on the target label.

In [27]:
encoder = sklearn.preprocessing.OneHotEncoder(handle_unknown = "ignore")
encoder.fit(x_resampled[categorical])
x = encoder.transform(x_resampled[categorical]) # train set
xx = encoder.transform(x_test[categorical]) # test set

this encoder returns a one column array which we change to an array that we can trun into a datframe. additionally it only turns the categorical features and drops the others by dfault. to avoid any errors, I feed the categorical features for encoding and then concatenate them with the numerical ones. in this process, the encoded data is reindexed and for concatenating I set its index to the same one as before. otherwise we would have NaN values because of the non matching indexes.

the columns should also be converted to the same type to feed to the model.

In [28]:
train_one_hot = pandas.concat([pandas.DataFrame(x.toarray()).set_index(x_resampled.index), x_resampled[numerical]], axis = 1)
test_one_hot = pandas.concat([pandas.DataFrame(xx.toarray()).set_index(x_test.index), x_test[numerical]], axis = 1)

train_one_hot.columns = train_one_hot.columns.astype(str)
test_one_hot.columns = test_one_hot.columns.astype(str)

In [29]:
train_one_hot.shape, test_one_hot.shape

((11974, 85), (6028, 85))

we can see that the features are more than before and it has worked.

### Final Dataframe
This dataframe is the one we will work with to find the unfairness. it contains the features and the target label as **ground truth** and three more columns for each classification we do.

In [30]:
final_df = x_test
final_df["ground_truth"] = y_test

In [31]:
final_df.shape

(6028, 14)

In [32]:
final_df.head()

,age,workclass,fnlwgt,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,ground_truth
6793,75,Private,188612,9,Widowed,Adm-clerical,Not-in-family,White,0,0,0,40,United-States,0
29412,52,Private,206862,9,Married-civ-spouse,Craft-repair,Husband,White,1,0,0,40,United-States,0
7,74,State-gov,88638,16,Never-married,Prof-specialty,Other-relative,White,0,0,3683,20,United-States,1
9117,51,Private,146574,14,Married-civ-spouse,Exec-managerial,Husband,White,1,0,0,50,United-States,1
247,35,Private,272019,7,Married-civ-spouse,Craft-repair,Husband,White,1,0,2057,40,United-States,0


-----------------------------


## Applying machine learning and classifying the data

I just applied three different classifiers and calculated the accuracy for them. After each classification, I add the predicted labels to the fianl dataframe.

### SVM classification

In [33]:
svm = sklearn.svm.SVC()
svm_clf = svm.fit(train_one_hot, y_resampled)
svm_y = svm.predict(test_one_hot)

svm_accuracy = sklearn.metrics.accuracy_score(y_test, svm_y)
svm_accuracy

0.7850033178500332

Adding the column for the predicted labels by this classifier.

In [34]:
final_df["svm_y"] = svm_y

### Decision tree classification

In [35]:
tree = sklearn.tree.DecisionTreeClassifier()
tree_clf = tree.fit(train_one_hot, y_resampled)
tree_y = tree.predict(test_one_hot)

tree_accuracy = sklearn.metrics.accuracy_score(y_test, tree_y)
tree_accuracy

0.7669210351692104

Adding the column for the predicted labels by this classifier.

In [36]:
final_df["tree_y"] = tree_y

### KNN classification

In [37]:
knn = sklearn.neighbors.KNeighborsClassifier()
knn_clf = knn.fit(train_one_hot, y_resampled)
knn_y = knn.predict(test_one_hot)

knn_accuracy = sklearn.metrics.accuracy_score(y_test, knn_y)
knn_accuracy

0.6045122760451228

Adding the column for the predicted labels by this classifier.

In [38]:
final_df["knn_y"] = knn_y

this is our final dataframe to work with.

In [39]:
final_df.head()

,age,workclass,fnlwgt,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,ground_truth,svm_y,tree_y,knn_y
6793,75,Private,188612,9,Widowed,Adm-clerical,Not-in-family,White,0,0,0,40,United-States,0,0,1,1
29412,52,Private,206862,9,Married-civ-spouse,Craft-repair,Husband,White,1,0,0,40,United-States,0,0,1,0
7,74,State-gov,88638,16,Never-married,Prof-specialty,Other-relative,White,0,0,3683,20,United-States,1,0,1,0
9117,51,Private,146574,14,Married-civ-spouse,Exec-managerial,Husband,White,1,0,0,50,United-States,1,0,1,1
247,35,Private,272019,7,Married-civ-spouse,Craft-repair,Husband,White,1,0,2057,40,United-States,0,0,0,0


Now we have a dataframe with
- a **sensitiv feature**: sex
- our target label or **ground truth**
- and three different classifications

---------------------------------


## Unfiarness Detection

we need male and female values separately, so I out them into different dataframes.


In [40]:
female_df = final_df[final_df["sex"] == 0]
male_df = final_df[final_df["sex"] == 1]

### Detecting **Disparate Impact**

Here we can find it with comparing the fraction of positive and negative predictions between male and female. we can find it using a group by on sepaprated dataframes and count the fraction f each group.

In [41]:
svm_female_positive = (female_df["svm_y"].sum()) / (female_df["sex"].count())
svm_female_positive

0.022314478463933574

In [42]:
svm_male_positive = (male_df["svm_y"].sum()) / (male_df["sex"].count())
svm_male_positive

0.0487685930260912

In [43]:
tree_female_positive = (female_df["tree_y"].sum()) / (female_df["sex"].count())
tree_female_positive

0.1883757135443695

In [44]:
tree_male_positive = (male_df["tree_y"].sum()) / (male_df["sex"].count())
tree_male_positive

0.44745184101438673

The fraction of positive predictions for males is more than the female. this shows unfairness in terms of *disparate impact* in both the classifications.

### Detecting **Disparate Mistreatment**

here we find if there is any difference between the accuracy of feamle subjects ond the male ones.

In [45]:
svm_accuracy_female = sklearn.metrics.accuracy_score(female_df["ground_truth"], female_df["svm_y"])
svm_accuracy_female

0.9024390243902439

In [46]:
svm_accuracy_male = sklearn.metrics.accuracy_score(male_df["ground_truth"], male_df["svm_y"])
svm_accuracy_male

0.7298219946354547

In [47]:
tree_accuracy_female = sklearn.metrics.accuracy_score(female_df["ground_truth"], female_df["tree_y"])
tree_accuracy_female

0.8546964193046186

In [48]:
tree_accuracy_male = sklearn.metrics.accuracy_score(male_df["ground_truth"], male_df["tree_y"])
tree_accuracy_male

0.7256766642282371

as it is obvious both of the models work better on female data. so we can conclude that it is biased against male. it could also be a result of overfitting.

### Detecting **Disparate Treatment**
to see if we have this kind of unfairness, we need to find rows that have everything other than the sensitive feature exactly the same. so we drop the predicted labels and the sencsitive feature, then find the rows that are the same.

In [49]:
disparate_df = final_df.drop(columns = ["sex", "svm_y",	"knn_y", "tree_y"], axis = 1)

In [50]:
duplicate_rows = disparate_df[disparate_df.duplicated(keep = False)]
duplicate_rows

,age,workclass,fnlwgt,education.num,marital.status,occupation,relationship,race,capital.gain,capital.loss,hours.per.week,native.country,ground_truth


there are no such rows. this could be the reason that we are only focused on **sex** as sensitive feature, but if the sensitive feature was all the ones that are important (e.g. race, marital status,...) we could have detect it.

### Detection using **KNN**

having the total number of individuals in each class and compare the ratio of male and feamle in them leads us to the result.

In [51]:
knn_df_count = pandas.DataFrame(final_df.groupby(by = "knn_y").count())
knn_df_count

,age,workclass,fnlwgt,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,ground_truth,svm_y,tree_y
knn_y,,,,,,,,,,,,,,,,
0,3431,3431,3431,3431,3431,3431,3431,3431,3431,3431,3431,3431,3431,3431,3431,3431
1,2597,2597,2597,2597,2597,2597,2597,2597,2597,2597,2597,2597,2597,2597,2597,2597


In [52]:
knn_df_sum = pandas.DataFrame(final_df.groupby(by = "knn_y").sum("sex"))
knn_df_sum

,age,fnlwgt,education.num,sex,capital.gain,capital.loss,hours.per.week,ground_truth,svm_y,tree_y
knn_y,,,,,,,,,,
0,128703,666026567,34143,2264,401536,66999,136907,653,1,1064
1,102495,478618629,26939,1837,6317738,475528,109240,866,242,1134


In [53]:
male_ratio_0 = (knn_df_sum["sex"].iloc[0])/(knn_df_count["sex"].iloc[0])
print(f"the male fraction in lower incmoe group is: {male_ratio_0}")

male_ratio_1 = (knn_df_sum["sex"].iloc[1])/(knn_df_count["sex"].iloc[1])
print(f"the male fraction in higher incmoe group is: {male_ratio_1}")

the male fraction in lower incmoe group is: 0.6598659283007869
the male fraction in higher incmoe group is: 0.7073546399691952


this ratio shows some amount of unfairness but it is not too high. the ratio in both classes are close to each other.

------------------------------


# Conclusion

We cleaned, trained, tested and detected unfairness in our dataset.

In this dataset we have worked based on only one sensitive, feature. But obviously it can be a set of sensitive features that is effecting the learning process. it can be **sex, race, native.country, marital.status, relationship** and other features combined.
with finding rules and checking their support and confidence we can find other sensitive features.

Resampling method used here is undersmapling which is because I did not want to detect unfairness in artificailly generated data. in real-life datasets we might work with the data without resampling and trying to make it more fair after finding the unfairnss.

I have also deleted the null values (for the same reason above) which could have been filled with the mode of the features.

---------------------------




# References


* imbalanced learn website (2023) 'RandomUnderSampler'. Available at: https://imbalanced-learn.org/stable/references/generated/imblearn.under_sampling.RandomUnderSampler.html#imblearn.under_sampling.RandomUnderSampler (Accessed: 10.09.2023)
* Kaggle website (2023) 'Adult income dataset'. Available at: https://www.kaggle.com/datasets/wenruliu/adult-income-dataset (Accessed: 27.08.2023)
* Scikit learn website (2023) 'sklearn.model_selection'. Available at: https://scikit-learn.org/stable/model_selection.html (Accessed: 10.09.2023)
* Stack Over Flow website (2023) Available at: https://stackoverflow.com/ (Accessed: 13.09.2023)

In [54]:
!jupyter nbconvert --to html Unfairness Detection Notebook.ipynb

[NbConvertApp] WARNING | pattern 'Unfairness' matched no files
[NbConvertApp] WARNING | pattern 'Detection' matched no files
[NbConvertApp] WARNING | pattern 'Notebook.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True